In [1]:
import json

word_data = json.loads(open('words.json').read())
state_data = json.loads(open('final.json').read())

In [2]:
steps = {}
for data in [word_data, state_data]:
    for k in data:
        d = data[k]
        p = d['participant']
        t = d['task']
        if (p,t) not in steps:
            steps[(p,t)] = [float('inf'), -float('inf')]
        steps[(p,t)][0] = min(d['start_step'], steps[(p,t)][0])
        steps[(p,t)][1] = max(d['end_step'], steps[(p,t)][1])

In [3]:
aligned = {}
for p,t in steps:
    mn, mx = steps[(p,t)]
    aligned[(p,t)] = [[None, None] for _ in range(mn, mx+1)]

In [6]:
for k in word_data:
    d = word_data[k]
    p = d['participant']
    t = d['task']
    start, end = steps[(p,t)]
    for i in range(d['start_step'], d['end_step']):
        aligned[(p,t)][int(i-start)][0] = d
        
for k in state_data:
    d = state_data[k]
    p = d['participant']
    t = d['task']
    start, end = steps[(p,t)]
    for i in range(d['start_step'], d['end_step']):
        aligned[(p,t)][int(i-start)][1] = d

In [14]:
out = open('aligned_data.tsv', 'w')
for p, t in aligned:
    start, end = steps[(p,t)]
    last = None
    for i, d in enumerate(aligned[(p, t)]):
        if d[0] is not None:
            word_lst = [d[0]['word'], d[0]['lemma'], d[0]['pos']]
        else:
            word_lst = ['NA', 'NA', 'NA']
        if d[1] is not None:
            state_lst = ['%s'%d[1]['z'], '%s'%d[1]['start_obj'], '%s'%d[1]['end_obj']]
        else:
            state_lst = ['NA', 'NA', 'NA']
        desc = '%s\t%s'%('\t'.join(word_lst), '\t'.join(state_lst))
        if not desc == last:
            out.write("%s\t%s\t%s\t%s\n"%(p, t, start+i, desc))
            last = desc
out.close()